# Dogs vs. Cats Redux: Kernels Edition

## 2. 모델구성 및 학습

In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import time
import re
import model_vgg
import train

/home/shkim/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 2.1 Model class
VGG19

### 2.2 이미지 파일 리스트 가져 오기

In [2]:
def getFiles(dir, word):
    files =   [dir+i for i in os.listdir(dir) if word in i]
    files.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    return files

In [3]:
def getDogCatData(image_dir, test_size=0.3):
    dog_files =  getFiles(image_dir, 'dog')
    cat_files =  getFiles(image_dir, 'cat')
    print("dog_files: ", len(dog_files))
    print("cat_files: ", len(cat_files))
    
    dog_files = dog_files[:200]
    cat_files = cat_files[:200]

    dog_cat_files = dog_files + cat_files

    label_dog = [0 for i in range(len(dog_files))]
    label_cat = [1 for i in range(len(cat_files))]
    label_one_hot = np.eye(2)[label_dog + label_cat]

    return train_test_split (dog_cat_files, label_one_hot, test_size=test_size, random_state=42)

In [4]:
DATA_DIR = "../data/"
TRAIN_DIR = DATA_DIR + "train_resize/"

x_train, x_valid, y_train, y_valid = getDogCatData (TRAIN_DIR)

print("train: ", len(x_train))
print("valid: ", len(x_valid))

dog_files:  12500
cat_files:  12500
train:  280
valid:  120


### 2.3 학습데이터와 valid 데이터 분리

In [5]:
learning_rate = 0.0001
epochs = 20
batch_size = 10

In [6]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
model = model_vgg.Model_Vgg19(sess, "model")
model.build_net(150)

Instructions for updating:
Use the retry module or similar alternatives.


In [7]:
train_dataset = train.DataSet2(sess, [x_train, y_train], batch_size = batch_size)
valid_dataset = train.DataSet2(sess, [x_valid, y_valid], batch_size = batch_size)
train_dataset.start()
valid_dataset.start()

In [8]:
train.train (model, train_dataset, valid_dataset, learning_rate, epochs)

train_count:  280 total_batch_train:  29
valid_count:  120 total_batch_valid:  13
Learning started. It takes sometime.


TypeError: Fetch argument None has invalid type <class 'NoneType'>

In [ ]:
train.eval (model, train_dataset, epochs)

In [ ]:
train.eval (model, valid_dataset, epochs)

In [ ]:
train_dataset.stop()
valid_dataset.stop()